# Labeling Algorithms

This notebook walks through segments of the PyMBE code base that pertain to generating labels from elements when there is not a clear user name but rather the label should be assembled from other attributes of the element and/or its owned elements.

The approach for complicated labels is to look at label fragments and how they can be combined in order to generate an integrated string.